In [1]:
from scipy.io import arff
import pandas as pd
data = arff.loadarff('seismic-bumps.arff')
dataset = pd.DataFrame(data[0])
datainfo = pd.DataFrame(data[1])
# print(dataset)
# print(datainfo)


In [2]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy
# from sklearn.model_selection import train_test_split
# integer encode
label_encoder = LabelEncoder()
dataset.iloc[:,0] = label_encoder.fit_transform(dataset.iloc[:,0])
dataset.iloc[:,1] = label_encoder.fit_transform(dataset.iloc[:,1])
dataset.iloc[:,2] = label_encoder.fit_transform(dataset.iloc[:,2])
dataset.iloc[:,7] = label_encoder.fit_transform(dataset.iloc[:,7])
dataset.iloc[:,-1] = label_encoder.fit_transform(dataset.iloc[:,-1])

X = dataset.iloc[:,:-1]

y = dataset.iloc[:,-1]

X = X.values
y = y.values

# print(y)
print("dataset_size: ", y.size)
# X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2)

dataset_size:  2584


In [3]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)

from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X, y, cv=10)

print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


[0.78764479 0.88030888 0.89189189 0.92664093 0.93410853 0.93410853
 0.93410853 0.93023256 0.93023256 0.93410853]
Accuracy: 0.91 (+/- 0.09)


In [4]:
from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(clf, X, y, cv=10)

from sklearn.metrics import confusion_matrix
print("confusion_matrix: ",confusion_matrix(y,y_pred))

from sklearn.metrics import precision_score,recall_score
print("precision: ",precision_score(y,y_pred))
print("recall:", recall_score(y,y_pred))

confusion_matrix:  [[2335   79]
 [ 158   12]]
precision:  0.13186813186813187
recall: 0.07058823529411765


In [5]:
#oversampling
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler()
# X_over, y_over = ros.fit_resample(X, y)

from imblearn.over_sampling import SMOTE
ros = SMOTE()
X_over, y_over = ros.fit_resample(X, y)

print("dataset_size: ", y_over.size)

scores_over = cross_val_score(clf, X_over, y_over, cv=10)

print(scores_over)
print("Accuracy of Oversampling: %0.2f (+/- %0.2f)" % (scores_over.mean(), scores_over.std() * 2))

y_pred_over = cross_val_predict(clf, X_over, y_over, cv=10)

print("confusion matrix of oversampling: ",confusion_matrix(y_over,y_pred_over))

print("precision of oversampling: ",precision_score(y_over,y_pred_over))
print("recall of oversampling:", recall_score(y_over,y_pred_over))

Using TensorFlow backend.


dataset_size:  4828
[0.69214876 0.70247934 0.74380165 0.85950413 0.86307054 0.86514523
 0.82572614 0.80912863 0.81950207 0.84232365]
Accuracy of Oversampling: 0.80 (+/- 0.12)
confusion matrix of oversampling:  [[1720  694]
 [ 261 2153]]
precision of oversampling:  0.7562346329469617
recall of oversampling: 0.891880695940348


In [6]:
#undersampling
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids()
X_under, y_under = cc.fit_resample(X, y)

print("dataset_size: ", y_under.size)

scores_under = cross_val_score(clf, X_under, y_under, cv=10)

print(scores_under)
print("Accuracy of Undersampling: %0.2f (+/- %0.2f)" % (scores_under.mean(), scores_under.std() * 2))

y_pred_under = cross_val_predict(clf, X_under, y_under, cv=10)

print("confusion matrix of undersampling: ",confusion_matrix(y_under,y_pred_under))

print("precision of undersampling: ",precision_score(y_under,y_pred_under))
print("recall of undersampling:", recall_score(y_under,y_pred_under))

dataset_size:  340
[0.44117647 0.35294118 0.47058824 0.47058824 0.58823529 0.70588235
 0.58823529 0.85294118 0.55882353 0.58823529]
Accuracy of Undersampling: 0.56 (+/- 0.27)
confusion matrix of undersampling:  [[95 75]
 [74 96]]
precision of undersampling:  0.5614035087719298
recall of undersampling: 0.5647058823529412


In [7]:
#combine oversampling and undersampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN()
X_combine, y_combine = smote_enn.fit_resample(X, y)

print("dataset_size: ", y_combine.size)

scores_combine = cross_val_score(clf, X_combine, y_combine, cv=10)

print(scores_combine)
print("Accuracy of combining oversampling and undersampling: %0.2f (+/- %0.2f)" % (scores_combine.mean(), scores_combine.std() * 2))

y_pred_combine = cross_val_predict(clf, X_combine, y_combine, cv=10)

print("confusion matrix of combining oversampling and undersampling: ",confusion_matrix(y_combine,y_pred_combine))

print("precision of combining oversampling and undersampling: ",precision_score(y_combine,y_pred_combine))
print("recall of combining oversampling and undersampling:", recall_score(y_combine,y_pred_combine))

dataset_size:  3157
[0.82649842 0.93690852 0.93037975 0.98417722 0.98734177 0.98412698
 0.97460317 0.98412698 0.96507937 0.98412698]
Accuracy of combining oversampling and undersampling: 0.96 (+/- 0.09)
confusion matrix of combining oversampling and undersampling:  [[1287  128]
 [  12 1730]]
precision of combining oversampling and undersampling:  0.9311087190527448
recall of combining oversampling and undersampling: 0.9931113662456946


In [8]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf_fs = ExtraTreesClassifier(n_estimators=50)
clf_fs = clf_fs.fit(X_combine, y_combine)
model_fs = SelectFromModel(clf_fs, prefit=True)
X_fs = model_fs.transform(X_combine)
print("feature dimension:", X_fs.shape[1])             

y_fs = y_combine

scores_fs = cross_val_score(clf, X_fs, y_fs, cv=10)

print(scores_fs)
print("Accuracy of feature selection: %0.2f (+/- %0.2f)" % (scores_fs.mean(), scores_fs.std() * 2))

y_pred_fs = cross_val_predict(clf, X_fs, y_fs, cv=10)

print("confusion matrix of feature selection: ",confusion_matrix(y_fs,y_pred_fs))

print("precision of feature selection: ",precision_score(y_fs,y_pred_fs))
print("recall of feature selection:", recall_score(y_fs,y_pred_fs))

feature dimension: 8
[0.70347003 0.73817035 0.78481013 0.89556962 0.87025316 0.8952381
 0.84126984 0.84761905 0.7968254  0.88253968]
Accuracy of feature selection: 0.83 (+/- 0.13)
confusion matrix of feature selection:  [[1037  378]
 [ 173 1569]]
precision of feature selection:  0.8058551617873652
recall of feature selection: 0.9006888633754305


In [9]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC(C=0.5, penalty="l1", dual=False).fit(X_combine, y_combine)
model_fs_2 = SelectFromModel(lsvc, prefit=True)
X_fs_2 = model_fs_2.transform(X_combine)
print("feature dimension:", X_fs_2.shape[1])
y_fs_2 = y_combine

scores_fs_2 = cross_val_score(clf, X_fs_2, y_fs_2, cv=10)

print(scores_fs_2)
print("Accuracy of feature selection: %0.2f (+/- %0.2f)" % (scores_fs_2.mean(), scores_fs_2.std() * 2))

y_pred_fs_2 = cross_val_predict(clf, X_fs_2, y_fs_2, cv=10)

print("confusion matrix of feature selection: ",confusion_matrix(y_fs_2,y_pred_fs_2))

print("precision of feature selection: ",precision_score(y_fs_2,y_pred_fs_2))
print("recall of feature selection:", recall_score(y_fs_2,y_pred_fs_2))

feature dimension: 14


//anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[0.83596215 0.7318612  0.91139241 0.93037975 0.87974684 0.88571429
 0.86984127 0.86984127 0.82857143 0.8984127 ]
Accuracy of feature selection: 0.86 (+/- 0.11)
confusion matrix of feature selection:  [[1095  320]
 [ 109 1633]]
precision of feature selection:  0.8361495135688684
recall of feature selection: 0.9374282433983927
